In [69]:
from xml.dom.minidom import parse
import xml.sax
import time

In [80]:
city_doc = parse('irece.osm')
def parser(content):
  city_info = []
  for node in city_doc.getElementsByTagName('node'):
    arr = []
    f = 0
    for tag in node.getElementsByTagName('tag'):
      if tag.getAttribute('k') == 'amenity':
        lat = node.getAttributeNode('lat').firstChild.data
        lon = node.getAttributeNode('lon').firstChild.data
        arr.append(tag.getAttribute('v'))
        arr.append(lat)
        arr.append(lon)
        f = 1
      if tag.getAttribute('k') == 'name' and f == 1:
        arr.append(tag.getAttribute('v'))
        info = {'name' : arr[3], 'type': arr[0], 'lat': arr[1], 'lon': arr[2]}
        city_info.append(info)
  return city_info

start_time = time.time()
city_info = parser(city_doc)
end_time = time.time()
elapsed_dom_parser = (end_time - start_time)
print("--- %s seconds ---" % elapsed_dom_parser)

--- 0.038271188735961914 seconds ---


In [75]:
city_info

[{'name': 'Restaurante João Dourado',
  'type': 'restaurant',
  'lat': '-11.3037224',
  'lon': '-41.8581656'},
 {'name': 'Bradesco',
  'type': 'bank',
  'lat': '-11.3029732',
  'lon': '-41.8572450'},
 {'name': 'Farma MiX',
  'type': 'pharmacy',
  'lat': '-11.3031678',
  'lon': '-41.8584142'},
 {'name': 'DIREC',
  'type': 'school',
  'lat': '-11.3036733',
  'lon': '-41.8560591'},
 {'name': 'Caixa Econômica Federal',
  'type': 'bank',
  'lat': '-11.3029000',
  'lon': '-41.8556083'},
 {'name': 'Correios',
  'type': 'post_office',
  'lat': '-11.3083862',
  'lon': '-41.8569280'},
 {'name': 'UFBA',
  'type': 'school',
  'lat': '-11.3073605',
  'lon': '-41.8571962'},
 {'name': 'Escola Rui Barbosa',
  'type': 'school',
  'lat': '-11.3020385',
  'lon': '-41.8502219'},
 {'name': 'Colegio Odete Nunes Dourado',
  'type': 'school',
  'lat': '-11.3059348',
  'lon': '-41.8662628'},
 {'name': 'Restaurante Prakasa Dia e Noite',
  'type': 'restaurant',
  'lat': '-11.3021755',
  'lon': '-41.8480638'},
 {

In [81]:
class Listener(xml.sax.ContentHandler):
  
  def __init__(self):    
    self.city_info = []
    self.name = ''
    self.type = ''
    self.lat = ''
    self.lon = ''
    self.serialize = False
    self.isAmenity = False

  def startElement(self, tag, attributes):        
    if tag == 'node':
      if 'lat' in attributes.getNames() and 'lon' in attributes.getNames():
        self.lat = attributes.get('lat')
        self.lon = attributes.get('lon')
    if tag == 'tag':
      if 'k' in attributes.getNames():
        if attributes.get('k') == 'amenity':
          self.type = attributes.get('v')
          self.isAmenity = True
        if attributes.get('k') == 'name' and self.isAmenity:
          self.name = attributes.get('v')
          self.serialize = True
          
  def restart(self):
    self.name = ''
    self.type = ''
    self.lat = ''
    self.lon = ''
    self.serialize = False
    self.isAmenity = False
        
  def endElement(self, tag):    
    if tag =='node':
      if len(self.lat) > 0 and len(self.lon) > 0:
        if self.serialize == True:
          self.city_info.append({'name' : self.name, 'type': self.type, 'lat': self.lat, 'lon': self.lon})
      self.restart()      
      
parser =  xml.sax.make_parser()

Handler = Listener()
parser.setContentHandler(Handler)
print("Starting SAX Parser...")

start_time = time.time()
parser.parse('irece.osm')
end_time = time.time()
elapsed_sax_parser = (end_time - start_time)
print("--- %s seconds ---" % elapsed_sax_parser)
#difference = elapsed_sax_parser / elapsed_dom_parser
#print(difference)
#print(Handler.city_info)

Starting SAX Parser...
--- 0.15914225578308105 seconds ---


In [68]:
Handler.city_info == city_info

True

In [63]:
len(city_info)

11